# Node-based learning

In [ ]:
## Imports and setup

In [ ]:
import os
import sys

sys.path.append(os.getcwd())
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import networkx as nx
from sklearn.metrics import roc_curve, auc
import numpy as np
import data_utils
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, matthews_corrcoef

%matplotlib inline

In [ ]:
fn_coauthors = "ca-HepPh.txt.gz"
df_coauthors = (
    data_utils.load_dataset_from_web(fn_coauthors)
    .rename(columns={0: "source", 1: "target"})
    .astype(str)
)
df_coauthors

## Introduction

In [ ]:
g = nx.from_pandas_edgelist(df_coauthors, "source", "target")
print(
    f"The raw graph has {g.number_of_nodes():,} nodes and {g.number_of_edges():,} edges"
)
g = data_utils.get_connected_component_subgraphs(g)[0]
print(
    f"The largest connected component that has {g.number_of_nodes():,} nodes and {g.number_of_edges():,} edges"
)

In [ ]:
# log-log plot of the degree distribution
fig, ax = plt.subplots()
log_degrees = np.log(np.array([d for n, d in g.degree]))
log_counts = np.log(pd.Series(log_degrees).value_counts().sort_index())
log_freq = log_counts / log_counts.sum()
ax.plot(log_counts.index, log_freq, ".")
fig.suptitle("Log-log plot of the degree distribution\n(the full dataset)")

For demonstration purposes, we will use very small percentage of the edges.

In [ ]:
df_sample = df_coauthors.sample(frac=0.01, random_state=42)
g = nx.from_pandas_edgelist(df_sample, "source", "target")
print(
    f"The raw graph has {g.number_of_nodes():,} nodes and {g.number_of_edges():,} edges"
)
g = data_utils.get_connected_component_subgraphs(g)[0]
layout = nx.spring_layout(g)
nx.draw(g, layout, node_size=10, node_color="black", edge_color="gray", alpha=0.5)
print(
    f"We are taking the largest connected component that has {g.number_of_nodes():,} nodes and {g.number_of_edges():,} edges"
)

In [ ]:
# log-log plot of the degree distribution
fig, ax = plt.subplots()
log_degrees = np.log(np.array([d for n, d in g.degree]))
log_counts = np.log(pd.Series(log_degrees).value_counts().sort_index())
log_freq = log_counts / log_counts.sum()
ax.plot(log_counts.index, log_freq, ".")
fig.suptitle("Log-log plot of the degree distribution\n(the 1-percent edge sample)")

The graph that we have here is a graph of co-authorship relationships. The nodes are authors, and the edges are co-authorship relationships. This graph was build by parsing author names from the abstract text, which may itroduce some ambiguity, mainly due to the fact that the same author may be referred to in different ways ("Gorelik B vs. Gorelik Boris"). Also, many authors may have the same name, which may lead to the same name being used for different authors. 

The dataset that we have is anonymous, we will simulate the latter form of ambiguity by scrambling some of the nodes

In [ ]:
# now, we will introduce some abiguity in the graph
np.random.seed(42)
all_author_ids = set(df_coauthors["source"]).union(set(df_coauthors["target"]))
frac_noise = 0.005
n_noise = int(frac_noise * len(all_author_ids))

scrambled = dict()

for i in range(n_noise):
    n_to_scramble = np.random.randint(2, 4)
    scrambled_name = f"scrambled_{i:02d}"
    nodes_to_scramble = []
    while len(nodes_to_scramble) < n_to_scramble:
        node = np.random.choice(list(all_author_ids))
        if node not in scrambled:
            scrambled[node] = scrambled_name
            nodes_to_scramble.append(node)

In [ ]:
df_scrambled = df_sample.copy()
df_scrambled["source"] = df_scrambled["source"].apply(lambda x: scrambled.get(x, x))
df_scrambled["target"] = df_scrambled["target"].apply(lambda x: scrambled.get(x, x))
g_scrambled = nx.from_pandas_edgelist(df_scrambled, "source", "target")
# add `scrambled` attribute to the nodes
for node in g_scrambled.nodes:
    g_scrambled.nodes[node]["scrambled"] = node.startswith("scrambled")
g_scrambled = data_utils.get_connected_component_subgraphs(g_scrambled)[0]
print(
    f"The largest connected component that has {g_scrambled.number_of_nodes():,} nodes and {g_scrambled.number_of_edges():,} edges"
)

In [ ]:
layout = nx.spring_layout(g_scrambled)
node_colors = [
    "red" if g_scrambled.nodes[n]["scrambled"] else "black" for n in g_scrambled.nodes
]
nx.draw(
    g_scrambled,
    layout,
    node_size=10,
    node_color=node_colors,
    edge_color="gray",
    alpha=0.5,
)
nx.write_graphml(g_scrambled, "/Users/boris/temp/graph_scrambled.graphml")

## Building a classifier to distinguish between the original and the scrambled nodes

To build a classifier that can distinguish between the original and the scrambled nodes, we will use the following features:
- the degree of the node
- the clustering coefficient of the node
- the number of triangles the node is part of
- the number of triangles the node is part of that are not part of any other triangle
- node centrality measures: betweenness, closeness, eigenvector


In [ ]:
def compute_node_features(g) -> pd.DataFrame:
    # approx 2 minutes of computation time on the full dataset
    degrees = dict(g.degree)
    clustering = nx.clustering(g)
    triangles = nx.triangles(g)
    triangles_no_others = dict()
    n_depth_2 = dict()
    n_depth_4 = dict()
    for node in tqdm(g.nodes, total=len(g.nodes), leave=False):
        triangles_no_others[node] = sum(
            [1 for n in g.neighbors(node) if n in triangles and triangles[n] == 1]
        )
        n_depth_2[node] = len(
            set(nx.single_source_shortest_path_length(g, node, cutoff=2))
        )
        n_depth_4[node] = len(
            set(nx.single_source_shortest_path_length(g, node, cutoff=4))
        )
    ret = pd.DataFrame(
        {
            "degree": degrees,
            "clustering": clustering,
            "triangles": triangles,
            "triangles_no_others": triangles_no_others,
            "n_depth_2": n_depth_2,
            "n_depth_4": n_depth_4,
        }
    )
    return ret


# df_node_features = compute_node_features(g_scrambled)

### Centrality features

Node centrality is a measure of the importance or influence of a node within a network. There are several types of centrality measures, such as:

- **Degree Centrality**: The number of direct connections a node has.
- **Closeness Centrality**: How close a node is to all other nodes in the network.
- **Betweenness Centrality**: The number of times a node acts as a bridge along the shortest path between two other nodes.
- **Eigenvector Centrality**: The influence of a node based on the influence of its neighbors.
- **Katz Centrality**: A measure that considers the number of all paths that lead to a node, attenuated by the length of the paths.
- **PageRank**: A measure that evaluates the importance of nodes based on the structure of incoming links.
- **HITS (Hubs and Authorities)**: Identifies two types of nodes: hubs, which point to many nodes, and authorities, which are pointed to by many hubs.
- **Clustering Coefficient**: The degree to which nodes tend to cluster together.


Note, that some of these metrics can be very computationally expensive, especially when computed directly, without any approximation heuristics.  


Look at the description of the betweenness centrality. It is a logical assumption that a node that represents multiple actual authors will have a higher betweenness centrality than a node that represents a single author. This is because the former will be part of many shortest paths between other nodes, while the latter will be part of fewer shortest paths. For the same reason, the closeness centrality of a node that represents multiple authors will be lower than the closeness centrality of a node that represents a single author.


In [ ]:
def compute_centrality_features(g) -> pd.DataFrame:
    # approx 8 minutes of computation time on the full dataset.
    # 1 minute on a 1-percent edge sample
    betweenness_centrality = nx.betweenness_centrality(g)
    closeness_centrality = nx.closeness_centrality(g)
    pagerank = nx.pagerank(g)
    ret = pd.DataFrame(
        {
            "betweenness_centrality": betweenness_centrality,
            "closeness_centrality": closeness_centrality,
            "katz": nx.katz_centrality(g),
        }
    )
    return ret


df_centrality_features = compute_centrality_features(g_scrambled)

### Community features

We may think of a community as a group of nodes that are more connected to each other than to the rest of the network. Community detection is the process of identifying these groups of nodes. These algorithms rely on various parameters such as resolution and are stochastic in their nature. Thus, we may assume that if a single node represents multiple authors, it is more likely that its community assignment will be stronger influenced by the changes in parameters. 


A good reading on community detection algorithms can be found here: https://memgraph.com/blog/community-detection-algorithms-with-python-networkx


#### Louvain community detection algorithm

We will use Louvain community detection algorithm to detect communities in the graph. The Louvain algorithm is a popular community detection algorithm that optimizes **Modularity**.
 
Modularity is a measure of the quality of a partition of a network into communities. It compares the number of edges within a community to the expected number of edges in a random network with the same degree distribution. The Louvain algorithm is based on the concept of modularity optimization, which is a measure of the quality of a partition of a network into communities. 

The algorithm works by iteratively optimizing the modularity of the network by moving nodes between communities.  The **resolution** parameter of the Louvain algorithm controls the trade-off between the size of the communities and the number of communities. A larger resolution parameter will result in fewer, larger communities, while a smaller resolution parameter will result in more, smaller communities.

In [ ]:
# community detection using louvain algorithm
fig, ax = plt.subplots()
for res in tqdm([0.1, 0.5, 1, 4]):
    communities = nx.community.louvain_communities(g_scrambled, resolution=res)
    sizes = [len(c) for c in communities]
    sizes.sort(key=lambda x: -x)
    ax.plot(range(len(sizes)), sizes, ".", label=f"resolution={res}")
    # assign `community_{res}` attribute to the nodes
    for i, c in enumerate(communities):
        for n in c:
            g_scrambled.nodes[n][
                f"community_{res:.2f}"
            ] = f"community@{res:.2f}_{i:03d}"
ax.legend()
ax.set_yscale("log")

What we see is that the larger the `resolution` parameter of the Louvain algorithm, the larger the communities that are detected. This is because the resolution parameter controls the trade-off between the size of the communities and the number of communities. A larger resolution parameter will result in fewer, larger communities, while a smaller resolution parameter will result in more, smaller communities.


We will use this use this information to count the fraction of neighbors that are in the same community as the node itself. We expect that scrambled nodes will have a lower fraction of neighbors in the same community.
 

We will repeat this for different values of the resolution parameter and use this as a feature for the classifier.

In [ ]:
def compute_community_features(g) -> pd.DataFrame:
    # about 1 minute of computation time
    resolutions = np.round(np.linspace(0.01, 4, 10), 2)
    for res in tqdm(resolutions):
        communities = nx.community.louvain_communities(g, resolution=res)
        sizes = [len(c) for c in communities]
        sizes.sort(key=lambda x: -x)
        # assign `community_{res}` attribute to the nodes
        for i, c in enumerate(communities):
            for n in c:
                g.nodes[n][f"community_{res:.2f}"] = f"community@{res:.2f}_{i:03d}"

    ret = []
    for n in g.nodes:
        neighbors = set(g.neighbors(n))
        curr = {"node_id": n}
        for res in resolutions:
            community_this = g.nodes[n][f"community_{res:.2f}"]
            neighbors_in_community = [
                g.nodes[nb][f"community_{res:.2f}"] == community_this
                for nb in neighbors
            ]
            neighboring_communities = set(
                [g.nodes[nb][f"community_{res:.2f}"] for nb in neighbors]
            )
            n_neighboring_communities = len(neighboring_communities)
            curr[f"same_community@{res:.2f}"] = sum(neighbors_in_community) / len(
                neighbors
            )
            curr[f"n_neighboring_communities@{res:.2f}"] = n_neighboring_communities
        ret.append(curr)
    df_community_features = pd.DataFrame(ret).set_index("node_id")
    cols_same = [c for c in df_community_features.columns if "same_community" in c]
    cols_neighboring = [
        c for c in df_community_features.columns if "n_neighboring" in c
    ]
    mean_same = df_community_features[cols_same].mean(axis=1)
    mean_neighboring = df_community_features[cols_neighboring].mean(axis=1)
    df_community_features["mean_same"] = mean_same
    df_community_features["mean_neighboring"] = mean_neighboring
    for perc in [10, 25, 75, 90]:
        df_community_features[f"percentile_same_{perc}"] = df_community_features[
            cols_same
        ].apply(lambda x: np.percentile(x, perc), axis=1)
        df_community_features[f"percentile_neighboring_{perc}"] = df_community_features[
            cols_neighboring
        ].apply(lambda x: np.percentile(x, perc), axis=1)

    return df_community_features


df_community_features = compute_community_features(g_scrambled)

### Ego-graph related features

An **ego graph** is a subgraph that includes a central node and all of its neighbors. Ego graphs are useful for analyzing the local structure of a network around a specific node. Sometimes it is intersting to examine the effect of removing the central node from the ego graph. We may assume that the effect of such a removal is stronger for nodes that represent multiple authors, than for nodes that represent a single author.

In the most obvious case, when a scrambled node connects two unrelated parts of a graph, removing it will disconnect the graph. But other, more subtle effects are possible, such as changes in the clustering coefficient, the diameter of the graph, or the number of connected components.

In [ ]:
def compute_ego_graph_features(g):
    vals = []
    for n in tqdm(g.nodes, desc="Ego-related features", total=len(g.nodes)):
        ego = nx.ego_graph(g, n, radius=1)
        ego_no_self = ego.copy()
        ego_no_self.remove_node(n)

        curr = dict(node_id=n)
        betweenness_ego = nx.betweenness_centrality(ego)
        curr["betweenness_ego"] = betweenness_ego[n]
        closeness_ego = nx.closeness_centrality(ego)
        curr["closeness_ego"] = closeness_ego[n]
        clustering_before = nx.average_clustering(ego)
        clustering_after = nx.average_clustering(ego_no_self)
        curr["delta_clustering_ego"] = clustering_after - clustering_before

        diameter_before = nx.diameter(ego)

        try:
            diameter_after = nx.diameter(ego_no_self)
        except nx.NetworkXError:
            # if the graph is not connected, the diameter is infinite
            # we'll set it to the number of nodes + 1
            diameter_after = len(ego_no_self.nodes) + 1

        curr["delta_diameter_ego"] = diameter_after - diameter_before

        curr["n_components_no_self_ego"] = nx.number_connected_components(ego_no_self)
        density_before = nx.density(ego)
        density_after = nx.density(ego_no_self)
        curr["delta_density_ego"] = density_after - density_before
        vals.append(curr)
    return pd.DataFrame(vals).set_index("node_id")


df_egograph_features = compute_ego_graph_features(g_scrambled)

### Supervised learning

Now, we have computed the features that we will use to build a classifier that can distinguish between the original and the scrambled nodes. We will use the following features:

In [ ]:
df_all = pd.concat(
    [df_egograph_features, df_centrality_features, df_community_features], axis=1
)
y = np.array(["scrambled" in i for i in df_community_features.index])
print(f"{df_all.shape=}, {y.mean()=:.3f}")

Our dataset is highly imbalanced. To force the classifier to pay more attention to the minority class, we will use sample weights. We will set the sample weight to 100 for the minority class and 1 for the majority class.

In [ ]:
weights = np.ones_like(y, dtype=float)
weights[y] = 100

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    df_all, y, weights, test_size=0.2, random_state=42
)

# Initialize the CatBoost classifier
classifier = CatBoostClassifier(
    iterations=1000,
    learning_rate=1e-4,
    depth=8,
    loss_function="Logloss",
    eval_metric="AUC",
    early_stopping_rounds=150,  # Early stopping
    l2_leaf_reg=10,  # L2 regularization
    subsample=0.8,  # Subsample fraction
    verbose=True,
)
# Fit the model with sample weights
classifier.fit(
    X_train, y_train, sample_weight=weights_train, eval_set=(X_test, y_test), plot=True
)

In [ ]:
def show_prediction_performance(X_test, y_test, classifier):
    # Predict and evaluate if needed
    y_pred = classifier.predict(X_test)
    p_pred_proba = classifier.predict_proba(X_test)[:, 1]
    mcc = matthews_corrcoef(y_test, y_pred)
    print(f"Matthews correlation coefficient: {mcc:.2f}")
    precision = 100 * np.sum(y_test & y_pred) / np.sum(y_pred)
    recall = 100 * np.sum(y_test & y_pred) / np.sum(y_test)
    print(
        f"Of the {y_pred.sum():3,d} nodes predicted as scrambled,      {precision:.1f}% are actually scrambled"
    )
    print(
        f"Of the {y_test.sum():,d}  nodes that are actually scrambled, {recall:.1f}% are predicted as scrambled"
    )

    fig, axes = plt.subplots(
        nrows=2, ncols=2, figsize=(9, 6), height_ratios=[5, 1], width_ratios=[5, 4]
    )
    ax = axes[0, 0]
    # roc curve
    fpr, tpr, _ = roc_curve(y_test, p_pred_proba)
    ax.plot(fpr, tpr)
    ax.plot([0, 1], [0, 1], linestyle="--")
    ax.set_xlabel("False positive rate")
    ax.set_ylabel("True positive rate")
    auc_score = auc(fpr, tpr)
    ax.set_title(f"AUC: {auc_score:.2f}", va="top")

    ax = axes[1, 0]
    lix = np.argsort(p_pred_proba)
    ax.plot(p_pred_proba[lix], ".", ms=0.5)
    y_test_sorted = y_test[lix]
    for i in range(len(y_test_sorted)):
        if y_test_sorted[i]:
            ax.axvline(i, color="red", alpha=0.4)
    ax.axhline(0.5, color="black", linestyle="--")

    ax = axes[0, 1]
    # feature importance
    try:
        feature_importance = (
            classifier.get_feature_importance(prettified=True).head(10).iloc[::-1]
        )
        feature_importance.plot.barh(
            x="Feature Id", y="Importances", legend=False, ax=ax
        )
        ax.set_ylabel("")
        ax.set_xlabel("Feature Importance")
    except:
        ax.axis("off")

    ax = axes[1, 1]
    # turn off
    ax.axis("off")
    fig.tight_layout()

In [ ]:
show_prediction_performance(X_test, y_test, classifier)
fig = plt.gcf()
fig.suptitle("Prediction performance on the partial dataset", y=1.02)

### Analyzing the full dataset
Now, let's load the full dataset

In [ ]:
df_full = data_utils.load_dataset_from_local("df_node_disambiguation_task").set_index(
    "node_id", drop=True
)
df_full.shape

In [ ]:
y = np.array(["scrambled" in i for i in df_full.index])
weights = np.ones_like(y, dtype=float)
weights[y] = 100
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    df_full, y, weights, test_size=0.2, random_state=42
)
classifier = CatBoostClassifier(
    iterations=1000,
    learning_rate=1e-5,
    depth=12,
    loss_function="Logloss",
    eval_metric="AUC",
    early_stopping_rounds=150,  # Early stopping
    l2_leaf_reg=10,  # L2 regularization
    subsample=0.8,  # Subsample fraction
    verbose=True,
)
classifier.fit(
    X_train, y_train, sample_weight=weights_train, eval_set=(X_test, y_test), plot=True
)

In [ ]:
show_prediction_performance(X_test, y_test, classifier)

#### SHAP Values for Better Interpretability

SHAP (SHapley Additive exPlanations) values provide a unified measure of feature importance. They are based on cooperative game theory and offer a way to explain the output of any machine learning model. SHAP values can be used to explain the prediction of any instance in terms of the contribution of each feature to the prediction.

SHAP values are computed by first considering all possible feature combinations and then calculating the contribution of each feature across these combinations. The values of the SHAP values represent the average contributions of the features to the prediction. SHAP values can be positive or negative, depending on whether the feature positively or negatively contributes to the prediction. The larger the absolute value of the SHAP value, the greater the feature's contribution to the prediction.

In [ ]:
import shap

shap_values = shap.TreeExplainer(classifier).shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, max_display=10)

You might be unimpressed by the performance of the classifier. However, consider that the only informtion that the classifier has is the structure of the graph. The classifier does not know the names of the nodes, the content of the abstracts, or any other information that could be used to distinguish between the original and the scrambled nodes. The fact that the classifier can distinguish between the original and the scrambled nodes with an AUC of 0.85 is quite impressive.

Compare the performance of the classifier with the performance of a majority class classifier and a random classifier.

In [ ]:
# for the purpose of performance comparison, we will use a majority class classifier and a random classifier
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train, sample_weight=weights_train)
show_prediction_performance(X_test, y_test, dummy)
fig = plt.gcf()
fig.suptitle("Majority class classifier", y=1.02)
dummy = DummyClassifier(strategy="prior")
dummy.fit(X_train, y_train, sample_weight=weights_train)
show_prediction_performance(X_test, y_test, dummy)
fig = plt.gcf()
fig.suptitle("Random classifier", y=1.02)